In [41]:
class Operation():
    
    def __init__(self, input_nodes = []):
       # Set up input nodes to the operation          
        self.input_nodes = input_nodes
        
       # where nodes are outputted to         
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
        
        _default_graph.operations.append(self)
    
    def compute(self):
        pass 
        

In [67]:
class add(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs=[x_var,y_var]
        return x_var + y_var
        
        
        
    

In [68]:
class multiply(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs=[x_var,y_var]
        return x_var * y_var
        
        
        


In [117]:
class matrix_multiply(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs=[x_var,y_var]
        return x_var.dot(y_var)
        
        
        


In [118]:
# Placeholder - an empty node that needs a value to be provided a compute output
# Variables - Changeable paramter of a graph
# Graph - global variables connecting variables and placeholders to operations

class PlaceHolder():
    def __init__(self):
        self.output_nodes = []
        _default_graph.placeholder.append(self)

In [119]:
class Variable():
    def __init__(self, initial_value = None):
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)
        

In [120]:
class Graph():
    
    def __init__(self):
        self.operatins = []
        self.variables = []
        self.placeholder = []
        
    def set_as_default(self):
        global _default_graph
        _default_graph = self

In [121]:
# This is the formula we will be following
z = Ax + b
A = 10
b = 1 

# Formula for a line
z = 10x + 1

SyntaxError: invalid syntax (<ipython-input-121-3905b3a9aac8>, line 7)

In [122]:
# create graph instance
g = Graph()

In [123]:
# set as default graph
g.set_as_default()

In [124]:
A = Variable(10)
b = Variable(1)

In [125]:
# will be provided by user
x = PlaceHolder()

In [126]:
y = multiply(A,x)

In [127]:
z = add(y,b)

In [128]:
# Now create post-order traveral to compute. 

def traverse_postorder(operation):
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node,Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
    recurse(operation)
    return nodes_postorder

In [145]:
import numpy as np

class Session():
    
    # Operation is the the operation we are computing aka the formula
    # feed_dict maps placeholder to input values
    def run(self,operation, feed_dict = {}):
        
        # called the posorder traversal on our operation
        nodes_postorder = traverse_postorder(operation)
        
        # Error checking
        print("data is", repr(nodes_postorder))
        
        # loop through each value in the list of nodes 
        for node in nodes_postorder:
            
            # check if value is a placeholder
            if type(node) == PlaceHolder:
                node.output = feed_dict[node]
            
            # check if value is a variable
            elif type(node) == Variable:
                node.output = node.value
            
            # we found an operation 
            else:
                # Operation - list of nodes from input_nodes
                node.inputs = [input_node.output for input_node in node.input_nodes]
                
               # we dont know how many inputs so we pass in args
               # here we compute the nodes passed in
                node.output = node.compute(*node.inputs)
            
        # converts list to numpy array
            if type(node.output) == list:
                node.output = np.array(node.output)
            
        return operation.output

In [146]:
sess = Session()

In [147]:
result = sess.run(operation = z,feed_dict = {x:10})

data is [<__main__.Variable object at 0x10b1346a0>, <__main__.PlaceHolder object at 0x10b134c18>, <__main__.matrix_multiply object at 0x10b1347f0>, <__main__.Variable object at 0x10b134208>, <__main__.add object at 0x10b13e6d8>]


In [148]:
# value after compute
# z = Ax + b
result

array([[15, 26],
       [35, 46]])

In [149]:
g = Graph()

In [150]:
g.set_as_default()

In [151]:
A = Variable([[1,2], [3,4]])

In [152]:
b = Variable([5,6])

In [153]:
x = PlaceHolder()

In [154]:
y = matrix_multiply(A,x)

In [155]:
z = add(y, b)

In [156]:
sess = Session()

In [157]:
sess.run(operation = z, feed_dict = {x:10})

data is [<__main__.Variable object at 0x10b21ec88>, <__main__.PlaceHolder object at 0x10b21e160>, <__main__.matrix_multiply object at 0x10b21e7b8>, <__main__.Variable object at 0x10b13e5f8>, <__main__.add object at 0x10b21e898>]


array([[15, 26],
       [35, 46]])